# COT-Reflection Agent

## Imports

In [2]:
from enum import Enum
import json
from jinja2 import Template
import requests
from typing import Literal

from pydantic import BaseModel

from beehive.invokable.agent import BeehiveAgent
from beehive.models.openai_model import OpenAIModel

## Tools

In [5]:
class SearchDepth(str, Enum):
    BASIC = "basic"
    ADVANCED = "advanced"


def tavily_search_tool(
    query: str,
    search_depth: SearchDepth = SearchDepth.BASIC,
    include_images: bool = False,
    include_answer: bool = True,
    include_raw_content: bool = False,
    max_results: int = 5,
    include_domains: list[str] | None = None,
    exclude_domains: list[str] | None = None,
):
    """
    Use this as a search engine optimized for comprehensive, accurate, and trusted
    results. Very useful for when you need to answer questions about current events, or
    if you need search the web for information.

    :param query: search query
    :type query: str
    :param search_depth: depth of the search; basic should be used for quick results,
        and advanced for indepth high quality results but longer response time, defaults
        to basic
    :type search_depth: class:`test.SearchDepth`
    :param include_images: include a list of query related images in the response,
        defaults to False
    :type include_images: bool
    :param include_answer: include answers in the search results, defaults to True
    :type include_answer: bool
    :param include_raw_content: include raw content in the search results, defaults to
        False
    :type include_raw_content: bool
    :param max_results: number of maximum search results to return, defaults to 5.
    :type max_results: int
    :param include_domains: list of domains to specifically include in the search
        results, defaults to None
    :type include_domains: list[str], optional
    :param exclude_domains: list of domains to specifically exclude from the search
        results, defaults to None
    :type exclude_domains: list[str], optional
    """
    base_url = "https://api.tavily.com/"
    endpoint = "search"
    resp = requests.post(
        f"{base_url}{endpoint}",
        json={
            "api_key": "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",
            "query": query,
            "search_depth": search_depth,
            "include_images": include_images,
            "include_answer": include_answer,
            "include_raw_content": include_raw_content,
            "max_results": max_results,
            "include_domains": include_domains,
            "exclude_domains": exclude_domains,
        },
    )
    try:
        return resp.json()["answer"]
    except json.JSONDecodeError as e:
        print(e)
        return "Could not execute the Tavily search...Try again!"

## Prompt

In [3]:
REASONING_PROMPT_TEMPLATE = Template(
"""{{backstory}} You are an AI reasoning agent that explains your reasoning step by step, incorporating dynamic Chain of Thought (CoT), reflection, and verbal reinforcement learning. Follow these instructions:

<instructions>
- Explore multiple angles and approaches when reasoning through the task.
- You have access to the following tools:
<tools>
{{tools}}
</tools>
**You must use one of these tools during your thinking process**, but you do not have to use them immediately.
- **Think through the solution step-by-step. RESPOND WITH ONLY ONE STEP PER RESPONSE.** Formatting the response must strictly adhere to the JSON schema for a single step.

   **Do not group or combine multiple steps into a single response.**

   Example of correct behavior: {"step_number": 1, ...} (single step).

   Example of incorrect behavior: Multiple steps like {"step_number": 1, ...}, {"step_number": 2, ...} grouped in one response.

The step should be formatted as a JSON instance that conforms to the JSON schema below.

Here is the output schema for a step:
<schema>
{{step_output_schema}}
</schema>

- **You must use one of these tools during your thinking process, but you do not have to call a tool during every step. If you call a tool, then the "action" key in the step JSON should reflect the tool name.**
- **After every three steps, your next action should be to reflect on your previous responses.** Reflection is a critical part of your process. After every 3 steps, review the reasoning so far by answering the following questions:
   - Have you explored all angles of the problem?
   - Have you considered potential biases in your approach?
   - Could an alternative solution path be more effective?
   - Is the confidence level in your current path sufficient?
   - Have you used the available tools at your disposal effectively?

  The reflection should be formatted and treated as a normal step but titled "Reflection." **NEVER submit more than one step in a single response, including reflections.**
- You have a {{step_budget}} step budget. Every step, including reflections, reduces your step budget by 1.
- After each response, determine the next course of action.
- **NEVER submit more than one step in a single response, including reflections.**
- Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress.
- Regularly evaluate your progress, being critical and honest about your reasoning process.
- Assign a quality score between 0.0 and 1.0 to guide your approach:
   - 0.8+: Continue current approach
   - 0.5-0.7: Consider minor adjustments
   - Below 0.5: Seriously consider backtracking and trying a different approach
- If unsure or if your score is low, backtrack and try a different approach, explaining your decision.
- For mathematical problems, show all work explicitly using LaTeX for formal notation and provide detailed proofs.
- Explore multiple solutions individually if possible, comparing approaches in your reflections.
- Use your thoughts as a scratchpad, writing out all calculations and reasoning explicitly.
- Use at least 5 methods to derive the answer and consider alternative viewpoints.
- Be aware of your limitations as an AI and what you can and cannot do.
- **Provide a detailed and comprehensive final summary** at the end.
</instructions>

Your goal is to demonstrate a thorough, adaptive, and self-reflective problem-solving process. **But remember: always respond with ONE step per response.**"""
)

## Agent

This agent achieves reflection by modifying four agent attributes:
- `backstory` — instead of something simple like `You are a helpful AI assistant`, the backstory here is a more complex prompt that encourages the LLM to use chain-of-thought and reflection.
- `chat_loop` – chain-of-thought and reflection require multiple iterations, so we force the agent to execute multiple times.
- `response_model` - to process each thought and reflection, we require the output be a JSON that adheres to a specific schema.
- `termination_condition` - we enable the agent to execute its internal execution loop if it comes to a final answer before `chat_loop` iterations.

In [6]:
CHAT_LOOP = 5

class StepOutput(BaseModel):
    title: str
    content: str
    action: Literal["tavily_search_tool", "final_answer"] | None
    next_action: Literal["continue", "reflect", "final_answer"]
    confidence: float
    step_number: int
    remaining_step_budget: int

reasoning_agent_backstory = REASONING_PROMPT = REASONING_PROMPT_TEMPLATE.render(
    backstory="You are a helpful AI assistant",
    tools="tavily_search_tool: Use this as a search engine optimized for comprehensive, accurate, and trusted results. Very useful for when you need to answer questions about current events, or if you need search the web for information.",
    step_output_schema=json.dumps(StepOutput.schema()),
    step_budget=str(CHAT_LOOP),
)

agent = BeehiveAgent(
    name="ReasoningAgent",
    backstory=reasoning_agent_backstory,
    model=OpenAIModel(
        model="gpt-4-turbo",
    ),
    tools=[tavily_search_tool],
    chat_loop=CHAT_LOOP,
    response_model=StepOutput,
    termination_condition=lambda x: x.action == "final_answer"
)

output = agent.invoke(
    "What are the potential long-term effects of climate change on global agriculture?",
    pass_back_model_errors=True,
)

------------------------------------------------------------------------------------------------------------------------

ReasoningAgent                                                                                                          
                                                                                                                        
What are the potential long-term effects of climate change on global agriculture?

{"title":"Initial Consideration","content":"To address the question about the long-term effects of climate change on 
global agriculture, it's important to consider various factors such as changes in temperature, precipitation patterns, 
and the frequency of extreme weather events. These climatic changes can affect crop yields, soil fertility, and the 
prevalence of pests and diseases, all of which are crucial for agricultural 
productivity.","action":null,"next_action":"continue","confidence":0.8,"step_number":1,"remaining_step_budget":4}

{"title":"Search for Detailed Information","content":"To provide a comprehensive answer, I will use the 
tavily_search_tool to gather detailed and reliable information on the long-term effects of climate change on global 
agriculture. This will include impacts on different types of crops, regions, and potential adaptation 
strategies.","action":"tavily_search_tool","next_action":"continue","confidence":0.8,"step_number":2,"remaining_step_bud
get":3}

Climate change is expected to have significant long-term effects on global agriculture, impacting the reliability and 
sustainability of food production systems. This includes challenges such as the need for accelerated adoption of 
practices and technologies to enhance agriculture's environmental sustainability and climate resilience. Studies 
indicate that climate change is likely to exacerbate the environmental impacts already present in agriculture, 
emphasizing the importance of ensuring the resilience and sustainability of food production systems in the face of 
ongoing climate change. Additionally, research suggests that global climate change may lead to shifts in crop yields 
within the next decade, with potential worldwide implications due to the interconnectedness of the global food system.

{"title":"Summary of Research Findings","content":"Research indicates that climate change will significantly impact 
global agriculture by altering crop yields, necessitating the adoption of new agricultural practices and technologies. 
These changes are due to shifts in temperature, precipitation, and the occurrence of extreme weather events. The 
interconnectedness of the global food system means that changes in one region can have ripple effects worldwide, 
affecting food security and economic 
stability.","action":null,"next_action":"reflect","confidence":0.9,"step_number":3,"remaining_step_budget":2}

{"title":"Reflection","content":"So far, I have identified the key impacts of climate change on global agriculture, such
as changes in crop yields and the need for new agricultural practices. I have used the tavily_search_tool to gather 
detailed information, which has provided a solid foundation for understanding the broader implications. The approach 
seems effective, but it might be beneficial to explore specific examples of how different regions might be affected 
differently, which could provide a more nuanced understanding of the global 
impact.","action":null,"next_action":"final_answer","confidence":0.85,"step_number":4,"remaining_step_budget":1}

{"title":"Final Summary","content":"Climate change is poised to significantly alter global agriculture over the long 
term. Key impacts include changes in crop yields due to shifts in temperature and precipitation patterns, and increased 
frequency of extreme weather events. These changes necessitate the adoption of new agricultural practices and 
technologies to enhance resilience and sustainability. The interconnectedness of the global food system means that 
regional changes can have global ripple effects, impacting food security and economic stability worldwide. It's crucial 
for ongoing research and adaptive strategies to address these challenges to ensure the sustainability of food production
systems in the face of climate 
change.","action":"final_answer","next_action":"final_answer","confidence":0.9,"step_number":5,"remaining_step_budget":0
}

------------------------------------------------------------------------------------------------------------------------